In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [5]:
training_data = datasets.FashionMNIST(
    root = 'data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:31<00:00, 829776.74it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 286242.19it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:05<00:00, 815017.71it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<00:00, 2624243.68it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [8]:
# Create data loaders.
train_dataloader = DataLoader(training_data,batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]:{X.shape}")
    print(f"Shape of y:{y.shape} {y.dtype}")
    break


Shape of X [N, C, H, W]:torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cpu device


In [16]:
len(training_data.classes)

10

In [17]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)


In [21]:
def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred,y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:5d}/{size:5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):0.1f}%, Avg loss: {test_loss:8f}\n")
    

In [22]:
epochs = 3
for t in range (epochs):
    print(f"Epoch {t+1}\n------------------------------")
    train(train_dataloader,model, loss_fn, optimizer)
    test(test_dataloader,model,loss_fn)
print("Done!")

Epoch 1
------------------------------
loss: 2.185487  [   64/60000]
loss: 2.181074  [ 6464/60000]
loss: 2.134676  [12864/60000]
loss: 2.150167  [19264/60000]
loss: 2.121746  [25664/60000]
loss: 2.051332  [32064/60000]
loss: 2.084658  [38464/60000]
loss: 2.018659  [44864/60000]
loss: 2.013164  [51264/60000]
loss: 1.953873  [57664/60000]
Test Error: 
 Accuracy: 58.8%, Avg loss: 1.954406

Epoch 2
------------------------------
loss: 1.973227  [   64/60000]
loss: 1.954426  [ 6464/60000]
loss: 1.852280  [12864/60000]
loss: 1.894023  [19264/60000]
loss: 1.799303  [25664/60000]
loss: 1.730892  [32064/60000]
loss: 1.755742  [38464/60000]
loss: 1.661803  [44864/60000]
loss: 1.676110  [51264/60000]
loss: 1.564988  [57664/60000]
Test Error: 
 Accuracy: 61.6%, Avg loss: 1.587576

Epoch 3
------------------------------
loss: 1.638877  [   64/60000]
loss: 1.609838  [ 6464/60000]
loss: 1.463911  [12864/60000]
loss: 1.540603  [19264/60000]
loss: 1.420399  [25664/60000]
loss: 1.391625  [32064/60000]
l

In [28]:
classes = training_data.classes
classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [29]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: {predicted}, Actual: {actual}")

Predicted: Ankle boot, Actual: Ankle boot
